In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from shapely.geometry import Point, LineString
import nept

from loading_data import get_data
from utils_plotting import plot_errors, plot_over_space, make_animation

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "decode-checks")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
import info.r063d2 as r063d2
import info.r063d6 as r063d6
# infos = [r063d2]

from run import spike_sorted_infos, r063_infos
infos = spike_sorted_infos
# infos = r063_infos

In [ ]:
def load_decoded(info, pickle_filepath, binsize=12):
    decoded_filename = info.session_id + '_decoded_binsize' + str(binsize) + 'cm.pkl'
    pickled_decoded = os.path.join(pickle_filepath, decoded_filename)
    
    with open(pickled_decoded, 'rb') as fileobj:
        decoded = pickle.load(fileobj)
    
    return decoded

def load_decoded_shuffled(info, pickle_filepath, binsize=12):
    shuffled_filename = info.session_id + '_decoded-shuffled_binsize' + str(binsize) + 'cm.pkl'
    shuffled_decoded = os.path.join(pickle_filepath, shuffled_filename)
    
    with open(shuffled_decoded, 'rb') as fileobj:
        shuffled = pickle.load(fileobj)
        
    return shuffled

In [ ]:
binsize = 12

In [ ]:
savefig = True

In [ ]:
# proportion decoded
proportions_decoded = []
session_ids = []
for info in infos:
    session_ids.append(info.session_id)
    
    decoded = load_decoded(info, pickle_filepath, binsize=binsize)

    proportion_decoded = []
    for trial, n_timebins in zip(decoded["decoded"], decoded["n_timebins"]):
        proportion_decoded.append(trial.n_samples/n_timebins)
    proportions_decoded.append(np.mean(proportion_decoded))

filename = "proportion-decoded_binsize"+str(binsize)+"cm.png"
filepath = os.path.join(output_filepath, "proportion")
if not os.path.exists(filepath):
    os.makedirs(filepath)

y_pos = np.arange(len(session_ids))
plt.bar(y_pos, proportions_decoded, align='center', alpha=0.7)
plt.xticks(y_pos, session_ids, rotation=90, fontsize=10)
plt.ylabel('Proportion')
plt.title("Samples decoded with %d cm bins" % binsize)
plt.tight_layout()

if savefig:
    plt.savefig(os.path.join(filepath, filename))
    plt.close()
else:
    plt.show()

In [ ]:
proportions_decoded

In [ ]:
decoded["n_timebins"]

In [ ]:
n_decoded

In [ ]:
# Individual likelihoods/errors over space
for info in infos:
    xx, yy = np.meshgrid(info.xedges, info.yedges)
    
    decoded = load_decoded(info, pickle_filepath, binsize=binsize)

    filename = info.session_id+"-likelihoods_byactual-"+str(binsize)+"cm.png"
    filepath = os.path.join(output_filepath, "likelihoods")
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    title = info.session_id+" posterior"
    likelihood_byactual = plot_over_space(info, decoded["likelihoods"], decoded["actual"], title, os.path.join(filepath, filename))

    filename = info.session_id+"-errors_byactual-"+str(binsize)+"cm.png"
    filepath = os.path.join(output_filepath, "errors", "over-space")
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    title = info.session_id+" error"
    errors_byactual = plot_over_space(info, decoded["errors"], decoded["actual"], title, os.path.join(filepath, filename))

In [ ]:
# Combined errors
n_sessions = 0
combined_decoded_errors = []
combined_shuffled_errors = []
for info in infos:
    n_sessions += 1
    decoded = load_decoded(info, pickle_filepath, binsize=binsize)
    combined_decoded_errors.append(decoded["errors"])

    shuffled = load_decoded_shuffled(info, pickle_filepath, binsize=binsize)
    combined_shuffled_errors.append(shuffled["errors"])

filename = "combined-errors_binsize"+str(binsize)+"cm.png"
filepath = os.path.join(output_filepath, "errors")
if not os.path.exists(filepath):
    os.makedirs(filepath)

plot_errors(combined_decoded_errors, combined_shuffled_errors, n_sessions=n_sessions, 
            filename=os.path.join(filepath, filename))

In [ ]:
# Individual errors
for info in infos:
    decoded = load_decoded(info, pickle_filepath, binsize=binsize)

    shuffled = load_decoded_shuffled(info, pickle_filepath, binsize=binsize)

    filename = info.session_id+"-errors_binsize"+str(binsize)+"cm.png"
    filepath = os.path.join(output_filepath, "errors")
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    plot_errors([decoded["errors"]], [shuffled["errors"]], n_sessions=1, 
                filename=os.path.join(filepath, filename))

In [ ]:
# animation
for info in infos:
    decoded = load_decoded(info, pickle_filepath, binsize=binsize)
    
    filepath = os.path.join(output_filepath, "animations")
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    trial_idx = 2
    make_animation(info, decoded, trial_idx, filepath)

In [ ]:
binsizes = [2, 12, 30]
# Individual mean/median errors
for info in infos:
    mean_errors = []
    median_errors = []
    
    mean_errors_shuffled = []
    median_errors_shuffled = []
    
    for bins in binsizes:
        combine_errors = []
        combine_errors_shuffled = []
        
        decoded = load_decoded(info, pickle_filepath, binsize=bins)

        shuffled = load_decoded_shuffled(info, pickle_filepath, binsize=bins)
            
        for error in decoded["errors"]:
            combine_errors.extend(error)
        mean_errors.append(np.mean(combine_errors))
        median_errors.append(np.median(combine_errors))
        
        for error in shuffled["errors"]:
            combine_errors_shuffled.extend(error)
        mean_errors_shuffled.append(np.mean(combine_errors_shuffled))
        median_errors_shuffled.append(np.median(combine_errors_shuffled))
        
    filename = info.session_id+"-mean-errors.png"
    filepath = os.path.join(output_filepath, "errors", "average")
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        
    plt.plot(binsizes, mean_errors)
    plt.xlabel("Binsize (cm)")
    plt.ylabel("Mean error")
    plt.title(info.session_id+" mean decode error (cm)")
    plt.savefig(os.path.join(filepath, filename))
    plt.close()
    
    filename = info.session_id+"-median-errors.png"
    filepath = os.path.join(output_filepath, "errors", "average")
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    
    plt.plot(binsizes, median_errors)
    plt.xlabel("Binsize (cm)")
    plt.ylabel("Median error")
    plt.title(info.session_id+" median decode error (cm)")
    plt.savefig(os.path.join(filepath, filename))
    plt.close()

In [ ]:
def expand_line(start_pt, stop_pt, line, expand_by):
    """Expands shapely line into a zone.

    Parameters
    ----------
    start_pt : shapely.Point
    stop_pt : shapely.Point
    line : shapely.LineString
    expand_by : int or float

    Returns
    -------
    zone : shapely.Polygon

    """
    line_expanded = line.buffer(expand_by)
    zone = start_pt.union(line_expanded).union(stop_pt)

    return zone


def find_zones(info, remove_feeder, expand_by):
    """Finds zones from ideal trajectories.

    Parameters
    ----------
    info : shortcut module
    remove_feeder: boolean
    expand_by : int or float
        Amount to expand the line.

    Returns
    -------
    zone : dict
        With shapely.Polygon as values.
        Keys are u, shortcut, novel.

    """
    u_line = LineString(info.u_trajectory)
    shortcut_line = LineString(info.shortcut_trajectory)
    novel_line = LineString(info.novel_trajectory)

    feeder1 = Point(info.path_pts['feeder1'][0], info.path_pts['feeder1'][1]).buffer(expand_by*1.3)
    feeder2 = Point(info.path_pts['feeder2'][0], info.path_pts['feeder2'][1]).buffer(expand_by*1.3)

    u_zone = expand_line(Point(info.u_trajectory[0]), 
                         Point(info.u_trajectory[-1]), 
                         u_line, expand_by)
    shortcut_zone = expand_line(Point(info.shortcut_trajectory[0]), 
                                Point(info.shortcut_trajectory[-1]), 
                                shortcut_line, expand_by)
    novel_zone = expand_line(Point(info.novel_trajectory[0]), 
                             Point(info.novel_trajectory[-1]), 
                             novel_line, expand_by)

    zone = dict()
    zone['u'] = u_zone
    zone['shortcut'] = shortcut_zone.difference(u_zone)
    zone['shortcut'] = zone['shortcut'].difference(novel_zone)
    zone['novel'] = novel_zone.difference(u_zone)

    if remove_feeder:
        for feeder in [feeder1, feeder2]:
            zone['u'] = zone['u'].difference(feeder)
            zone['shortcut'] = zone['shortcut'].difference(feeder)
            zone['novel'] = zone['novel'].difference(feeder)

    return zone


def find_subset_zones(info, remove_feeder, expand_by):
    """Finds zones from ideal trajectories.

    Parameters
    ----------
    info : shortcut module
    remove_feeder: boolean
    expand_by : int or float
        Amount to expand the line.

    Returns
    -------
    zone : dict
        With shapely.Polygon as values.
        Keys are u, shortcut, novel.

    """
    u_line = LineString(info.u_trajectory)
    shortcut_line = LineString(info.shortcut_trajectory)
    novel_line = LineString(info.novel_trajectory)
    u_subset_line = LineString(info.u_segment)

    feeder1 = Point(info.path_pts['feeder1'][0], info.path_pts['feeder1'][1]).buffer(expand_by*1.3)
    feeder2 = Point(info.path_pts['feeder2'][0], info.path_pts['feeder2'][1]).buffer(expand_by*1.3)

    u_zone = expand_line(Point(info.u_trajectory[0]), 
                         Point(info.u_trajectory[-1]), 
                         u_line, expand_by)
    shortcut_zone = expand_line(Point(info.shortcut_trajectory[0]), 
                                Point(info.shortcut_trajectory[-1]), 
                                shortcut_line, expand_by)
    novel_zone = expand_line(Point(info.novel_trajectory[0]), 
                             Point(info.novel_trajectory[-1]), 
                             novel_line, expand_by)
    u_subset_zone = expand_line(Point(info.u_segment[0]),
                                Point(info.u_segment[-1]),
                                u_subset_line, expand_by)

    zone = dict()
    zone['u'] = u_subset_zone
    zone['shortcut'] = shortcut_zone.difference(u_zone)
    zone['shortcut'] = zone['shortcut'].difference(novel_zone)
    zone['novel'] = novel_zone.difference(u_zone)

    if remove_feeder:
        for feeder in [feeder1, feeder2]:
            zone['u'] = zone['u'].difference(feeder)
            zone['shortcut'] = zone['shortcut'].difference(feeder)
            zone['novel'] = zone['novel'].difference(feeder)

    return zone

In [ ]:
for info in infos:
    print(info.session_id)
    
    decoded = load_decoded(info, pickle_filepath, binsize=binsize)
    
    events, position, spikes, _, _ = get_data(info)

    binned_maze_shape = (len(info.yedges)-1, len(info.xedges)-1)
    xx, yy = np.meshgrid(info.xedges, info.yedges)

    zones = find_subset_zones(info, remove_feeder=True, expand_by=15)

    xcenters = info.xedges[:-1] + (info.xedges[1:] - info.xedges[:-1]) / 2
    ycenters = info.yedges[:-1] + (info.yedges[1:] - info.yedges[:-1]) / 2

    u_zone = np.zeros(binned_maze_shape).astype(bool)
    shortcut_zone = np.zeros(binned_maze_shape).astype(bool)
    novel_zone = np.zeros(binned_maze_shape).astype(bool)

    for i, x in enumerate(xcenters):
        for j, y in enumerate(ycenters):
            if zones["u"].contains(Point(x,y)):
                u_zone[j][i] = True
            elif zones["shortcut"].contains(Point(x,y)):
                shortcut_zone[j][i] = True
            elif zones["novel"].contains(Point(x,y)):
                novel_zone[j][i] = True

    sliced_position = position.time_slice(info.task_times["phase3"].start, info.task_times["phase3"].stop)
    occupancy = nept.get_occupancy(sliced_position, info.yedges, info.xedges)

    phase1_position = position.time_slice(info.task_times["phase1"].start, info.task_times["phase1"].stop)
    phase1_occupancy = nept.get_occupancy(phase1_position, info.yedges, info.xedges)
    phase2_position = position.time_slice(info.task_times["phase2"].start, info.task_times["phase2"].stop)
    phase2_occupancy = nept.get_occupancy(phase2_position, info.yedges, info.xedges)
    u_pos = np.zeros(binned_maze_shape).astype(bool)
    u_pos[occupancy > 0.] = True
    u_pos[phase1_occupancy > 0.] = True
    u_pos[phase2_occupancy > 0.] = True
    u_area = np.zeros(binned_maze_shape).astype(bool)
    u_area[u_pos & u_zone] = True

    shortcut_pos = np.zeros(binned_maze_shape).astype(bool)
    shortcut_pos[(occupancy > 0.) & (~u_area)] = True
    shortcut_area = np.zeros(binned_maze_shape).astype(bool)
    shortcut_area[shortcut_pos & shortcut_zone] = True

    novel_pos = np.zeros(binned_maze_shape).astype(bool)
    novel_pos[(occupancy > 0.) & (~u_area)] = True
    novel_area = np.zeros(binned_maze_shape).astype(bool)
    novel_area[novel_pos & novel_zone] = True

In [ ]:
novel_area.shape, xcenters.shape, ycenters.shape

In [ ]:
decoded["decoded"][0].x

In [ ]:
xcenters

In [ ]:
u_decoded = []
shortcut_decoded = []
novel_decoded = []
u_actual = []
shortcut_actual = []
novel_actual = []

for i in range(len(decoded["decoded"][0])):
    x_idx = nept.find_nearest_idx(decoded["decoded"][0].x[i], xcenters)
    y_idx = nept.find_nearest_idx(decoded["decoded"][0].y[i], ycenters)

    if novel_area[y_idx][x_idx]:
        novel_decoded.append([decoded["decoded"][0].x[i], decoded["decoded"][0].y[i], decoded["decoded"][0].time[i]])
    elif shortcut_area[y_idx][x_idx]:
    elif u_area[y_idx][x_idx]:

In [ ]:
combined = np.zeros(binned_maze_shape)
combined[u_area] = 1
combined[shortcut_area] = 2
combined[novel_area] = 3

plt.figure()
pp = plt.pcolormesh(xx, yy, combined, cmap="PuBuGn")
plt.plot(position.x, position.y, "k.", ms=2)
plt.plot(decoded["decoded"][0].x[i], decoded["decoded"][0].y[i], "r.", ms=10)
plt.tight_layout()
plt.show()